In [1]:
# Import required packages
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import xarray as xr
import rasterio as rio
import rioxarray
import geopandas as gpd
import time
from glob import glob
import scipy.signal

## Prep results for mintpy

In [ ]:
# all except for first appear to have identical spatial extent

In [2]:
ft_paths = glob('./proc_DT121/*/merged/range_offsets_phase.vrt')

In [29]:
rioxarray.open_rasterio(ft_paths[12])

<xarray.DataArray (band: 1, y: 1549, x: 6444)>
[9981756 values with dtype=float64]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 6.442e+03 6.442e+03 6.444e+03
  * y            (y) float64 1.548e+03 1.548e+03 1.546e+03 ... 2.5 1.5 0.5
    spatial_ref  int64 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [ ]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [ ]:
# write config file for mintpy
CONFIG_TXT = f'''# vim: set filetype=cfg:
##----------------------------- hyp3 ---------------------##
mintpy.load.processor        = isce
##---------interferogram datasets:
mintpy.load.unwFile          = {data_path}/*/merged/range_offsets_phase
mintpy.load.corFile          = {data_path}/*/merged/topophase.cor
##---------geometry datasets:
mintpy.load.demFile          = {data_path}/*/merged/dem.crop.vrt
mintpy.load.lookupYFile      = {data_path}/*/merged/lat.rdr.full.vrt
mintpy.load.lookupXFile      = {data_path}/*/merged/lon.rdr.full.vrt
mintpy.load.incAngleFile     = {data_path}/*/merged/los.rdr.full.vrt
mintpy.load.azAngleFile      =${data_path}/*/merged/los.rdr.full.vrt

mintpy.reference.lalo        = auto
mintpy.troposphericDelay.method  = no

mintpy.compute.cluster    = local
mintpy.compute.numWorker  = 6
''' 